<a href="https://colab.research.google.com/github/Ark1645/Skin-Disease-Detection/blob/main/Copy_of_Prediction_Using_Efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
import matplotlib.pylab as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.applications import mobilenet_v2
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root='/content/drive/MyDrive/Datasets/IMG_CLASSES'
img_height=224
img_width=224
BATCH_SIZE=32

In [ ]:
def prefucn(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=1 - 0.2, upper=1 + 0.2)
    image = tf.image.random_saturation(image, lower=1 - 0.2, upper=1 + 0.2)
    image = tf.image.random_hue(image, max_delta=0.2)
    return image

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=prefucn,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
    )

In [ ]:
train_generator = train_datagen.flow_from_directory(
    root,
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')


Found 24442 images belonging to 10 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    root,
    target_size=(img_width, img_height),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

Found 2711 images belonging to 10 classes.


In [ ]:
class_indices = train_generator.class_indices
print(class_indices)

{'1. Eczema 1677': 0, '10. Warts Molluscum and other Viral Infections - 2103': 1, '2. Melanoma 15.75k': 2, '3. Atopic Dermatitis - 1.25k': 3, '4. Basal Cell Carcinoma (BCC) 3323': 4, '5. Melanocytic Nevi (NV) - 7970': 5, '6. Benign Keratosis-like Lesions (BKL) 2624': 6, '7. Psoriasis pictures Lichen Planus and related diseases - 2k': 7, '8. Seborrheic Keratoses and other Benign Tumors - 1.8k': 8, '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k': 9}


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model.trainable = False

model = models.Sequential([
    base_model,
    # layers.GlobalAveragePooling2D(),
    # layers.Dense(128, activation='relu'),
    # layers.BatchNormalization(),
    # layers.Dropout(0.25),
    # layers.Dense(128, activation='relu'),
    # layers.BatchNormalization(),
    # layers.Dropout(0.25),
    # layers.Dense(64, activation='relu'),
    # layers.BatchNormalization(),
    # layers.Dropout(0.25),
    # layers.Dense(100, activation='softmax')  # Adjust this based on your dataset's number of classes
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 4062381 (15.50 MB)
Trainable params: 4020358 (15.34 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [ ]:
model.compile(Adam(learning_rate=6e-5 ), 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = 3,
    # callbacks=[checkpoint]
    )

Epoch 1/3
 45/763 [>.............................] - ETA: 3:20:18 - loss: 2.1185 - accuracy: 0.2681

In [ ]:
loss, accuracy = model.evaluate(val_gen)
print(f'Validation accuracy: {accuracy*100:.2f}%')

NameError: name 'model' is not defined